In [0]:
import re
import pandas as pd


In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [15]:
cd '/content/gdrive/My Drive/Illumio'

/content/gdrive/My Drive/Illumio


In [0]:
class Firewall:
  filepath = ''
  def __init__(self, path):
    self.filepath = path
  def accept_packet(self,direction,protocol,port, ip_add):
    info = []
    info.append(str(direction))
    info.append(str(protocol))
    info.append(str(port))
    info.append(str(ip_add))
    df = pd.read_csv(self.filepath)
    df.direction = df.direction.astype(str)
    df.protocol = df.protocol.astype(str)
    df.port = df.port.astype(str)
    df['IP address']= df['IP address'].astype(str)
    #filtering rules based on direction and protocol
    map_input = (df['direction'] == info[0]) & (df['protocol'] == info[1])
    df_filtered =  df[map_input]
    df_allfilter = df_filtered[(df_filtered['port'] == info[2]) & (df_filtered['IP address'] == info[3])]
    #check if input matches any rule exactly (without range)
    if df_allfilter.shape[0] > 0:
      return True
    #check if there exist rules with same direction and protocol as input. if not, there is not match
    if df_filtered is None:
      return False    
    #iterate rows which have same direction and protocol  
    for idx,row in df_filtered.iterrows():
      port_flag = 0
      ip_flag = 0
      #if port range exists in rule, check if input lies within the range
      if '-' in row['port']:
        port_ranges = row['port'].split('-')
        if (int(info[2]) >= int(port_ranges[0])) and (int(info[2]) <= int(port_ranges[1])):
          port_flag = 1
      #else, just check if they are equal
      else:
        if int(info[2]) == int(row['port']):
          port_flag = 1
      #if ip range exists, check if input ip address lies within the range
      if '-' in row['IP address']:
        ip_ranges = row['IP address'].split('-')
        ip_nums1 = ip_ranges[0].split('.')
        ip_nums2 = ip_ranges[1].split('.')
        ip_nums_check = info[3].split('.')
        for n,n1,n2 in zip(ip_nums_check,ip_nums1,ip_nums2):
          if (int(n) == int(n1)):
            ip_flag =1
            continue
          elif (int(n) >= int(n1)) and (int(n) <= int(n2)):
            ip_flag = 1
            continue  
          else:
            ip_flag = 0
      #else, just check if IP addresses match
      else:
        if row['IP address'] == info[3]:
          ip_flag = 1
      if (port_flag ==1) and ip_flag ==1:
        return True
    return False

In [0]:
f = Firewall('sample_rules.csv')

In [68]:
test_1 = f.accept_packet("inbound", "tcp", 80, "192.168.1.2")
print(test_1)

True


In [69]:
test_2 = f.accept_packet("inbound", "udp", 53, "192.168.2.1")
print(test_2)

True


In [70]:
test_3 = f.accept_packet("outbound", "tcp", 10234, "192.168.10.11")
print(test_3)

True


In [72]:
test_4 = f.accept_packet("inbound", "tcp", 81, "192.168.1.2")
print(test_4)

False


In [73]:
test_5 = f.accept_packet("inbound", "udp", 24, "52.12.48.92")
print(test_5)

False
